In [13]:
import torch
from db_helper_functions import get_stock_news_from_db
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)


In [14]:
df = get_stock_news_from_db("AAPL")
df = df[~df.article.isnull()]
t_df = df.iloc[:100]
t_df

,id,ticker,title,url,article,date
0,4235,AAPL,"Microsoft, Alphabet, Meta, Apple, And Amazon L...",https://www.benzinga.com/news/earnings/22/07/2...,(Monday Market Open) Investors appear optimist...,2022-07-25
1,4376,AAPL,Benzinga Before The Bell: More Amazon Workers ...,https://www.benzinga.com/news/22/08/28532545/b...,CNBCBlackRock To Pledge A$1B In Australian Bat...,2022-08-17
2,3,AAPL,"Market Rebounds On Trade Optimism, Tech Bounce...",https://www.benzinga.com/node/12934767,A stronger-than-expected government report on ...,2019-01-04
3,4,AAPL,American Shippers And Carriers React To Signs ...,https://www.benzinga.com/node/12938837,The first week of 2019 saw three of America's ...,2019-01-07
4,5,AAPL,TD Ameritrade IMX Continued Its Dip In Decembe...,https://www.benzinga.com/node/12953011,"Declining for the third month in a row, TD Ame...",2019-01-08
...,...,...,...,...,...,...
95,132,AAPL,46 Stocks Moving In Wednesday's Mid-Day Session,https://www.benzinga.com/node/13642737,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-01
96,133,AAPL,Mid-Afternoon Market Update: Nu Skin Enterpris...,https://www.benzinga.com/node/13643990,"Toward the end of trading Wednesday, the Dow t...",2019-05-01
97,135,AAPL,60 Biggest Movers From Yesterday,https://www.benzinga.com/node/13649199,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-02
98,137,AAPL,"After Sliding On Powell's Words, Stocks Look F...",https://www.benzinga.com/node/13652243,The Fed threw one right down the middle late W...,2019-05-02


In [15]:
finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone", force_download=True)

config.json: 100%|██████████| 533/533 [00:00<?, ?B/s] 
vocab.txt: 100%|██████████| 226k/226k [00:00<00:00, 13.7MB/s]


In [16]:
finbert_model = AutoModelForSequenceClassification.from_pretrained(
    "yiyanghkust/finbert-tone", num_labels=3
)

pytorch_model.bin: 100%|██████████| 439M/439M [00:18<00:00, 23.6MB/s] 


In [17]:
start_pos = 512
end_pos = 512 * 2
raw_list_data_for_finbert = [x[start_pos:end_pos] for x in t_df.article]

In [18]:
finbert_mat = finbert_tokenizer(
    raw_list_data_for_finbert, padding=True, return_tensors="pt"
)

In [19]:
outputs = finbert_model(**finbert_mat)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1706, -4.9509,  8.1077],
        [ 0.2905,  0.0974, -2.5024],
        [-5.9380, 10.0288, -5.7915],
        [-3.8902, -3.4970, 11.6473],
        [-2.8115, -3.2362,  7.6035],
        [-1.5745,  3.4449, -5.8679],
        [-2.8695,  1.9628, -0.7169],
        [ 4.5318, -5.4036, -1.0344],
        [-1.2308, -4.2488,  7.3972],
        [-5.1422, -0.7307,  9.0710],
        [-2.7236, -3.3387,  8.9899],
        [ 0.1838, -3.9975,  5.5663],
        [ 2.6510, -4.4525, -1.3548],
        [-4.6552, -2.3832, 10.8676],
        [-7.0718, 12.0662, -6.6328],
        [-6.0417,  8.7086, -5.7935],
        [-1.7315, -0.2952,  3.0724],
        [-1.9062,  3.0620, -3.3200],
        [-3.0855, -2.0258,  8.2393],
        [ 1.9825, -6.2243,  5.8269],
        [ 7.8742, -6.3236, -5.7039],
        [-1.1159, -2.4342,  6.5349],
        [-3.4164, -3.5709, 10.9312],
        [ 5.5782, -3.6536, -5.0314],
        [ 1.5906, -2.7137,  0.4595],
        [-4.2627, -0.0983,  4.36

In [20]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [21]:
t_df[["positive", "negative", "neutral"]] = np.array(predictions.tolist()).round(
    decimals=2
)
t_df

C:\Users\vince\AppData\Local\Temp\ipykernel_35256\3064107347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df[["positive", "negative", "neutral"]] = np.array(predictions.tolist()).round(
C:\Users\vince\AppData\Local\Temp\ipykernel_35256\3064107347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df[["positive", "negative", "neutral"]] = np.array(predictions.tolist()).round(
C:\Users\vince\AppData\Local\Temp\ipykernel_35256\3064107347.py:1: SettingWithCopyWarning: 
A value is trying to be set on

,id,ticker,title,url,article,date,positive,negative,neutral
0,4235,AAPL,"Microsoft, Alphabet, Meta, Apple, And Amazon L...",https://www.benzinga.com/news/earnings/22/07/2...,(Monday Market Open) Investors appear optimist...,2022-07-25,0.00,0.00,1.00
1,4376,AAPL,Benzinga Before The Bell: More Amazon Workers ...,https://www.benzinga.com/news/22/08/28532545/b...,CNBCBlackRock To Pledge A$1B In Australian Bat...,2022-08-17,0.53,0.44,0.03
2,3,AAPL,"Market Rebounds On Trade Optimism, Tech Bounce...",https://www.benzinga.com/node/12934767,A stronger-than-expected government report on ...,2019-01-04,0.00,1.00,0.00
3,4,AAPL,American Shippers And Carriers React To Signs ...,https://www.benzinga.com/node/12938837,The first week of 2019 saw three of America's ...,2019-01-07,0.00,0.00,1.00
4,5,AAPL,TD Ameritrade IMX Continued Its Dip In Decembe...,https://www.benzinga.com/node/12953011,"Declining for the third month in a row, TD Ame...",2019-01-08,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...
95,132,AAPL,46 Stocks Moving In Wednesday's Mid-Day Session,https://www.benzinga.com/node/13642737,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-01,0.00,1.00,0.00
96,133,AAPL,Mid-Afternoon Market Update: Nu Skin Enterpris...,https://www.benzinga.com/node/13643990,"Toward the end of trading Wednesday, the Dow t...",2019-05-01,0.00,1.00,0.00
97,135,AAPL,60 Biggest Movers From Yesterday,https://www.benzinga.com/node/13649199,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-02,0.00,1.00,0.00
98,137,AAPL,"After Sliding On Powell's Words, Stocks Look F...",https://www.benzinga.com/node/13652243,The Fed threw one right down the middle late W...,2019-05-02,0.94,0.00,0.06


In [23]:
idx_positive = t_df["positive"].idxmax()
idx_negative = t_df["negative"].idxmax()
idx_neutral = t_df["neutral"].idxmax()
most_positive = t_df.loc[idx_positive, "article"]
most_negative = t_df.loc[idx_negative, "article"]
most_neutral = t_df.loc[idx_neutral, "article"]
print(f"Most positive: {most_positive}")
print(f"Most negative: {most_negative}")
print(f"Most neutral: {most_neutral}")


Most positive: Few investors would argue 2018 was a good year for their portfolios. No one would argue that it was better than 2017. A historical look at TD Ameritrade’s Investor Movement Index (IMX) paints a fairly stark portrait of just how much worse last year was than its predecessor.Turns out, that gut check is more true than many might realize. The December 2018 release of the IMX, which tracks buying and selling behavior among the broker’s 11 million retail clients, posted its lowest score in more than two years at 4.41. That score is also within a tenth of its all-time low of 4.33, which it hit in March 2016.Compare that to the December 2017’s IMX report, which marked the index’s all-time high of 8.59.While there’s plenty of reasons underlying the difference in performance between the two years, the sheer rate of the drop each December bookends is worth closer scrutiny.First, A RecapEven a cursory glance at the historical IMX chart reveals the tide shifted quickly entering the 